In [ ]:
from imports import *

In [ ]:
# Define names and filenames...

galaxy = 'ngc1512'
galaxy_hst = galaxy
root_dir = '/Users/abarnes/Dropbox/work/Smallprojects/galaxies'

hstha_file = '%s/data_hstha/%s/hst_contsub/%s_hst_ha_sic.fits' %(root_dir, galaxy_hst, galaxy_hst)
muscat_file = '%s/data_hstha/%s/muse/%s_nebmask.fits' %(root_dir, galaxy_hst, galaxy.upper())
musha_file = '%s/data_hstha/%s/muse/%s-*_MAPS.fits' %(root_dir, galaxy_hst, galaxy.upper())
musha_file = glob(musha_file)[0] #because of resolution in name

cutout_dir = '%s/data_hstha_nebulae_catalogue/%s/cutouts' %(root_dir, galaxy_hst)
dendro_dir = '%s/data_hstha_nebulae_catalogue/%s/dendro' %(root_dir, galaxy_hst)
cutouts_hdus_dir = '%s/data_hstha_nebulae_catalogue/%s/cutouts_hdus' %(root_dir, galaxy_hst)

rerun_all = True
rerun_regions = True
rerun_masking = True

regions_file = '%s/sample.reg' %cutout_dir
regions_pickel_file = '%s/sample.pickel' %cutout_dir
sample_table_file = '%s/data_misc/sample_table/phangs_sample_table_v1p6.fits' %root_dir
muscat_table_file = '%s/data_misc/Nebulae_catalogue_v3/Nebulae_catalogue_v3.fits' %root_dir

print(hstha_file)
print(muscat_file)
print(musha_file)
print(cutout_dir)
print(dendro_dir)
print(cutouts_hdus_dir)
print(regions_file)
print(regions_pickel_file)
print(sample_table_file)
print(muscat_table_file)

In [ ]:
# Make paths 

def make_paths(rootdir, galaxy_hst, rerun_all=False):

    root_dir = '%s/data_hstha_nebulae_catalogue/%s/' %(rootdir, galaxy_hst)
    cutout_dir = '%s/cutouts' %(root_dir)
    dendro_dir = '%s/dendro' %(root_dir)
    cutouts_hdus_dir = '%s/cutouts_hdus' %(root_dir)

    print('[Info] Outputing to the following:')
    print(root_dir)

    if rerun_all: 
        os.system('rm -rf '+root_dir)

    if not os.path.isdir(root_dir):
        os.mkdir(root_dir) 

    for path in [cutout_dir, dendro_dir, cutouts_hdus_dir]:

        if not os.path.isdir(path):
            os.mkdir(path)  

make_paths(root_dir, galaxy_hst)

In [ ]:
# Loading files...

hstha_hdu = fits.open(hstha_file)[0]
musha_hdu = fits.open(musha_file)['HA6562_FLUX']
muscat_hdu = fits.open(muscat_file)[0]

# Update arrays
musha_hdu.data[np.isnan(musha_hdu.data)] = -100
muscat_hdu.data = np.array(muscat_hdu.data, dtype=float)
muscat_hdu.data[muscat_hdu.data==-1] = np.nan

# Convert all hdus to float32
hdus = [hstha_hdu, musha_hdu, muscat_hdu]
hdus = [cat_misc.convert_to_float32(hdu.copy()) for hdu in hdus]

In [ ]:
# Get regions and sample table... 

if os.path.exists(regions_file) & ~rerun_regions:
    regions = cat_misc.load_pickle(regions_pickel_file)
else: 
    muscat_table = cat_misc.get_museprops(galaxy, muscat_table_file)
    regions_sky = cat_cutouts.get_ds9regions_all(muscat_table, outputfile=regions_file)
    regions = cat_cutouts.get_regions(regions_file)
    cat_misc.save_pickle(regions, regions_pickel_file) 

In [8]:
# Get cutouts and save... 

hdus = [hstha_hdu, musha_hdu, muscat_hdu]
names = ['hstha_hdu', 'musha_hdu', 'muscat_hdu']

for hdu, name in zip(hdus, names):

    print('[INFO] Running for %s...' %name)
    hdu_cutouts = cat_cutouts.get_croppeddata_all(hdu, regions)
    cat_misc.save_pickle(hdu_cutouts, '%s/%s.pickel' %(cutout_dir, name)) 
    del hdu_cutouts
    _ = gc.collect()

Cropping regions: 100%|██████████| 632/632 [00:55<00:00, 11.32it/s]


[INFO] [save_pickle] Saved to /Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha_nebulae_catalogue/ngc1512/cutouts/hstha_hdu.pickel
[INFO] Running for musha_hdu...


Cropping regions: 100%|██████████| 632/632 [00:35<00:00, 17.70it/s]


[INFO] [save_pickle] Saved to /Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha_nebulae_catalogue/ngc1512/cutouts/musha_hdu.pickel
[INFO] Running for muscat_hdu...


Cropping regions: 100%|██████████| 632/632 [00:34<00:00, 18.06it/s]

[INFO] [save_pickle] Saved to /Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha_nebulae_catalogue/ngc1512/cutouts/muscat_hdu.pickel


In [9]:
# Load all cutouts and save in one file... 

names = ['hstha_hdu', 'musha_hdu', 'muscat_hdu']
hdus_cutouts = {}

for name in tqdm(names):

    hdu_cutouts = cat_misc.load_pickle('%s/%s.pickel' %(cutout_dir, name))
    hdus_cutouts[name] = hdu_cutouts

cat_misc.save_pickle(hdus_cutouts, '%s/hdus_all.pickel' %cutout_dir) 

 67%|██████▋   | 2/3 [00:00<00:00, 12.87it/s]

[INFO] [load_pickle] Load /Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha_nebulae_catalogue/ngc1512/cutouts/hstha_hdu.pickel
[INFO] [load_pickle] Load /Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha_nebulae_catalogue/ngc1512/cutouts/musha_hdu.pickel
[INFO] [load_pickle] Load /Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha_nebulae_catalogue/ngc1512/cutouts/muscat_hdu.pickel


100%|██████████| 3/3 [00:00<00:00, 15.98it/s]


[INFO] [save_pickle] Saved to /Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha_nebulae_catalogue/ngc1512/cutouts/hdus_all.pickel
